<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day12_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00


In [75]:
TEST = False #@param {type: "boolean"}
TEST_INPUT1 = 'day12_test1.txt'
TEST_INPUT2 = 'day12_test1.txt'
INPUT = 'day12_input1.txt'
PART=2 #@param {type: "integer"}

In [ ]:
import ipdb
import functools
import re
import time


def get_input() -> list[str]:
  if PART == 1:
    input = TEST_INPUT1 if TEST else INPUT
  else:
    input = TEST_INPUT2 if TEST else INPUT

  with open(input, 'r') as f:
    rows = f.read().splitlines()
  return rows


def parse_input(input: tuple[str], unfold: bool=False) -> list[tuple[str, tuple[int]]]:
  results = []

  parse_regex = r'^([\?\#\.]+) (([0-9]+,?)+)$'
  for input_row in input:
    m = re.match(parse_regex, input_row)
    row = m.group(1)
    groups = [int(c) for c in m.group(2).split(',')]

    # Need to make groups a tuple so that it is hashable for
    # when we memoize get_combinations.
    results.append((row, tuple(groups)))
  if unfold:
    results = [tuple([((r[0] + '?') * 4) + r[0], tuple(r[1]*5)]) for r in results]
  return results

# We're going to be doing *a lot* of recursion in step 2, with a high
# degree of cachability since unfolding repeats the row pattern and
# matching groups.
@functools.cache
def get_combinations(problem: tuple[str, tuple[int]]) -> int:
  row, groups = problem

  # Base cases
  if not groups:
    # 1. no more groups and no more #'s left in the row, so it's a match
    if '#' not in row:
      return 1
    # 2. no more groups, but we still have #'s, so no match.
    else:
      return 0
  # 3. groups remain, but not enough chars to match
  if len([c for c in row if c in ('?', '#')]) < groups[0]:
    return 0

  char_to_match = row[0]
  group_to_match = groups[0]

  match char_to_match:
    # If we find a #, there should be exactly enough to match the next group.
    # Any more or less than this is a mismatch.
    case '#':
      possible_match=row[:group_to_match]
      if all((c in ['?', '#']) for c in possible_match) and (
      group_to_match == len(row) or row[group_to_match] in ('.', '?')):
        # Strip off an extra character because it's either a '.' or
        # a '?' and if the latter we can only interpret as a '.' in
        # order for the groups to be separated
        return get_combinations((row[group_to_match+1:], groups[1:]))
      else:
        return 0
    # We don't care about dots, they are just fillers
    case '.':
      return get_combinations((row[1:], groups))
    # Expand both posibilities for ?
    case '?':
      return get_combinations(('#' + row[1:], groups)) + get_combinations(('.' + row[1:], groups))


start_time = time.time()
input = get_input()
problems = parse_input(input, False if PART == 1 else True)
answer = 0
for problem in problems:
  combinations = get_combinations(problem)
  print(f'{problem} -- {combinations}')
  answer += combinations
  print ('=' * 40)
print ('=' * 40)
print(f'answer: {answer}')
print(f'run time: {time.time() - start_time} (s)')